In [1]:
from prep_data import PrepData
from anchor_boxes import *
from yolo_model import *

import numpy as np
import tensorflow as tf

In [2]:
input_shape = (676, 380, 3)
grid_dim = 13
anchor_sizes = [(0.1, 0.1), (0.2, 0.2), (0.3, 0.3)]
num_anchors_persize = 2
num_anchors = num_anchors_persize * len(anchor_sizes)

In [3]:
anchor_boxes = create_anchor_boxes((grid_dim, grid_dim), num_anchors)

p = PrepData()
train, test = p.load_images()
labels = p.connect_labels(train, grid_dim, num_anchors, anchor_boxes, iou_anchor_label)
train, test = p.preprocess_images(train), p.preprocess_images(test)

In [4]:
# swap axes of x and y of the image
train = np.swapaxes(train, 1, 2)
test = np.swapaxes(test, 1, 2)
train.shape

(1001, 676, 380, 3)

In [13]:
def custom_loss(grid_dim, num_anchors):
    def loss(labels, predictions):
        errors = []

        for i in range(grid_dim):
            for j in range(grid_dim):
                predicted_boxes = labels[i][j] + predictions[i][j]
                
                # Calculate iou scores and loss for each anchor box
                for k in range(num_anchors):
                    box = predicted_boxes[k]
                    label_box = labels[i][j][k]
                    iou_score = iou_pred_label(box, label_box)
                    
                    confidence = predictions[i, j, k, 4]
                    print(confidence)
                    print(predictions[i, j, k])
                    errors.append(tf.abs(iou_score - confidence))

        # Return MSE of the errors
        return tf.reduce_mean(tf.square(errors))
    return loss

In [14]:
model = yolo_model(input_shape, grid_dim, num_anchors)
model.compile(optimizer='adam', loss=custom_loss(grid_dim, num_anchors), metrics=['accuracy'])

/Users/jadenvanrijswijk/Car Pose Estimation/.venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [15]:
model.fit(train, labels, epochs=10, batch_size=16)
model.evaluate(test, labels)

Epoch 1/10


NameError: name 'iou_pred_label' is not defined